In [1]:
from plot_params import *
import pandas as pd
df=pd.read_csv(outdir / 'Mouse_annotated.csv.gz')

In [2]:
df['subseq'].str.len().min()

30

In [3]:
df['subseq'].str.len().max()

30

In [4]:
df.head()

,Unnamed: 0,transcript_id,deletion_index_start,deletion_index_end,difference,synapse_probability,start_annotation,end_annotation,subseq
0,0,ENSMUST00000000001,0,30,0.001099,0.030640,UTR5,UTR5,CACACATCCGGTTCTTCCGGGAGCTAGGGG
1,1,ENSMUST00000000001,15,45,0.001587,0.031128,UTR5,UTR5,TCCGGGAGCTAGGGGAGCTGACGGAGAAGG
2,2,ENSMUST00000000001,30,60,-0.001343,0.028198,UTR5,UTR5,AGCTGACGGAGAAGGCCACCGCCCAGCAGA
3,3,ENSMUST00000000001,45,75,-0.001099,0.028442,UTR5,UTR5,CCACCGCCCAGCAGAAGACCCGTCTCCGCC
4,4,ENSMUST00000000001,60,90,0.002197,0.031738,UTR5,UTR5,AGACCCGTCTCCGCCGGTGTGTGGCGATTC


In [5]:
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Seq import Seq
def df2fasta(df, fname, col='subseq', id='Unnamed: 0'):
    
    
    sequences = []
    for index, row in df.iterrows():
        sequences.append(SeqRecord(Seq(row[col]), str(row[id]), "", ""))
    with open(fname, "w") as output_handle:
        SeqIO.write(sequences, output_handle, "fasta")


In [6]:
df2fasta(df, outdir / 'all.fa')

In [7]:
df['annotation']=df.apply(
    lambda row: 'UTR5' if row['start_annotation']=='UTR5' and row['end_annotation']=='UTR5' else
     'UTR3' if row['start_annotation']=='UTR3' and row['end_annotation']=='UTR3' else 'CDS', axis = 1)

In [8]:
df2fasta(df.loc[(df['annotation']=='UTR3')&(df['difference']<-0.1)], 
         outdir / 'utr3_forefround.fa')
df2fasta(df.loc[(df['annotation']=='UTR3')&(df['difference']>0.1)], 
         outdir / 'utr3_backfround.fa')

df2fasta(df.loc[(df['annotation']=='UTR5')&(df['difference']<-0.1)], 
         outdir / 'utr5_forefround.fa')
df2fasta(df.loc[(df['annotation']=='UTR5')&(df['difference']>0.1)], 
         outdir / 'utr5_backfround.fa')

In [9]:
df.loc[(df['annotation']=='UTR5')&(df['difference']<-0.1)]

,Unnamed: 0,transcript_id,deletion_index_start,deletion_index_end,difference,synapse_probability,start_annotation,end_annotation,subseq,annotation
4034,4034,ENSMUST00000000206,15,45,-0.181641,0.312500,UTR5,UTR5,GTCCCTCCAGTGGACTTAGGGACAGAGCAC,UTR5
7139,7139,ENSMUST00000000317,210,240,-0.633301,0.073730,UTR5,UTR5,TCTGAATCTTTGTAGCCGCATCCCCAAGCC,UTR5
7246,7246,ENSMUST00000000335,225,255,-0.173828,0.423828,UTR5,UTR5,AGACCTGTGCGCGGCCTTGACGAGGGGATG,UTR5
7247,7247,ENSMUST00000000335,240,270,-0.195312,0.402344,UTR5,UTR5,CTTGACGAGGGGATGAGAGAGTCCTACCAC,UTR5
44452,44452,ENSMUST00000002100,120,150,-0.121094,0.503906,UTR5,UTR5,GAGCCAGCCTGGAGGCCGGTCCCTGATCAC,UTR5
...,...,...,...,...,...,...,...,...,...,...
5690888,5690888,ENSMUST00000238943,510,540,-0.160156,0.742188,UTR5,UTR5,TGGTTTTTTCCTGCTCTGTCTTCTCTCTCC,UTR5
5693970,5693970,ENSMUST00000238972,1530,1560,-0.164062,0.597656,UTR5,UTR5,TGTTACAGGAGCACAGGACCATGCTGCCCA,UTR5
5693971,5693971,ENSMUST00000238972,1545,1575,-0.167969,0.593750,UTR5,UTR5,GGACCATGCTGCCCATCAGTTTTATGTGGA,UTR5
5693972,5693972,ENSMUST00000238972,1560,1590,-0.156250,0.605469,UTR5,UTR5,TCAGTTTTATGTGGACACTGGGACTGAACT,UTR5


# Commands to run HOMER

```
cd /tscc/nfs/home/hsher/ps-yeolab5/karen_data
findMotifs.pl utr3_forefround.fa fasta utr3_homer_results/ -fasta utr3_backfround.fa \
    -rna -nofacts
findMotifs.pl utr5_forefround.fa fasta utr5_homer_results/ -fasta utr5_backfround.fa \
    -rna -nofacts
```